In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

In [3]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.2.4
2.5


In [4]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [5]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [6]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [7]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

In [8]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'), function=<function sum_over at 0x00000207906B28B0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'), function=<function sum_over at 0x00000207906B28B0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'), function=<function sum_over at 0x00000207906B28B0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'), function=<function sum_over at 0

In [9]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{' 0_RfR 0':0,
                                  '0_RfR 1':0,
                                  '0_RfR 2':0,
                                  'A.3_DikeIncrease 0':0}),
           Policy('policy 2', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':0,
                                'A.3_DikeIncrease 0':5}),
           Policy('policy 3', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':0,
                                'A.3_DikeIncrease 0':10}),
            Policy('policy 4', **{'0_RfR 0':1,
                                 '0_RfR 1': 0,
                                '0_RfR 2': 0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 5', **{'0_RfR 0':0,
                                 '0_RfR 0':1,
                                '0_RfR 0':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 6', **{'0_RfR 0':0,
                                 '0_RfR 1':0,
                                '0_RfR 2':1,
                                'A.3_DikeIncrease 0':0}),
        Policy('policy 7', **{' 1_RfR 0':1,
                                  '1_RfR 1':0,
                                  '1_RfR 2':0,
                                  'A.3_DikeIncrease 0':0}),
           Policy('policy 8', **{'1_RfR 0':0,
                                 '1_RfR 1':1,
                                '1_RfR 2':0,
                                'A.5_DikeIncrease 0':0}),
           Policy('policy 9', **{'1_RfR 0':0,
                                 '1_RfR 1':0,
                                '1_RfR 2':1,
                                'A.3_DikeIncrease 0':0}),
            Policy('policy 10', **{'2_RfR 0':1,
                                 '2_RfR 1':0,
                                '2_RfR 2':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 11', **{'2_RfR 0':0,
                                 '2_RfR 1':1,
                                '2_RfR 2':0,
                                'A.3_DikeIncrease 0':0}),
          Policy('policy 12', **{'2_RfR 0':0,
                                 '2_RfR 1':0,
                                '2_RfR 2':1,
                                'A.3_DikeIncrease 0':0})
            ]

In [10]:
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

In [11]:
use_pickle1 = True
if use_pickle1:
    with open('data/formulation_results.pickle', 'rb') as filehandler:
        results = pickle.load(filehandler)
else:
    #pass the policies list to EMA workbench experiment runs
    n_scenarios = 100
    with MultiprocessingEvaluator(dike_model) as evaluator:
        results = evaluator.perform_experiments(n_scenarios, policies)
    # Save results in Pickle file
    with open('data/formulation_results.pickle', 'wb') as filehandler:
        pickle.dump(results, filehandler)


In [12]:
import pandas as pd
a,b = results
df = pd.DataFrame(b)
df["policy"] = pd.DataFrame(a)["policy"]
df

#a = np.min(df['A.3 Total Costs'])#



,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,policy
0,9.506030e+07,0.000000,1.118850e+08,0.002483,1.389003e+08,0.231006,2.485405e+07,0.004771,2.983051e+08,0.220732,0.0,0.0,policy 1
1,9.506030e+07,0.000000,7.323512e+08,0.478150,1.959433e+07,0.000000,1.686899e+07,0.000000,7.344192e+07,0.001258,0.0,0.0,policy 1
2,2.512252e+09,1.652505,1.093152e+08,0.000000,9.549722e+07,0.131235,1.686899e+07,0.000000,7.197799e+07,0.000000,0.0,0.0,policy 1
3,9.506030e+07,0.000000,1.260903e+08,0.017792,6.827920e+07,0.097941,4.256294e+07,0.013238,1.166882e+08,0.046773,0.0,0.0,policy 1
4,2.508217e+09,1.649032,1.093152e+08,0.000000,2.421301e+08,0.400379,4.272025e+07,0.011854,7.197799e+07,0.000000,0.0,0.0,policy 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,2.679112e+09,1.903257,1.022563e+08,0.000000,5.437894e+07,0.065691,1.623000e+07,0.000000,4.542000e+07,0.000000,30700000.0,0.0,policy 12
1196,2.508319e+09,1.885683,1.022563e+08,0.000000,1.858000e+07,0.000000,7.595766e+07,0.024761,4.542000e+07,0.000000,30700000.0,0.0,policy 12
1197,8.964065e+07,0.002453,1.070046e+08,0.004018,3.478393e+07,0.025415,4.141514e+07,0.011307,4.542000e+07,0.000000,30700000.0,0.0,policy 12
1198,3.159608e+08,0.177236,2.741476e+08,0.169799,1.858000e+07,0.000000,8.926443e+07,0.036158,4.542000e+07,0.000000,30700000.0,0.0,policy 12


In [13]:
#dit moet dus gebeuren per 100 scenarios
print(np.min(df['A.3 Total Costs']))
print(np.average(df['A.3 Total Costs']))
print(np.max(df['A.3 Total Costs']))
print(np.percentile(df['A.3 Total Costs'], 95))
print(np.percentile(df['A.3 Total Costs'], 5))

18580000.0
338919753.2389691
2785616795.6057982
1954454010.5283349
18580000.0


In [14]:
def q05(x):
    return x.quantile(0.05)
def q95(x):
    return x.quantile(0.95)

In [15]:
group_df = df.groupby(by="policy")
agg_par = ['mean', 'min', 'max', q05, q95]
agg_dict = {
    'A.3 Total Costs': agg_par,
    'A.3_Expected Number of Deaths': agg_par,
    'RfR Total Costs':agg_par,
    'Expected Evacuation Costs':agg_par
}
TCA_df = group_df.agg(agg_dict)
TCA_df

A.3 Total Costs                                            \
                     mean           min           max           q05   
policy                                                                
policy 1     3.881642e+08  1.858000e+07  2.719248e+09  1.858000e+07   
policy 10    2.537979e+08  1.858000e+07  2.298932e+09  1.858000e+07   
policy 11    2.537979e+08  1.858000e+07  2.298932e+09  1.858000e+07   
policy 12    2.537979e+08  1.858000e+07  2.298932e+09  1.858000e+07   
policy 2     7.336466e+07  5.077736e+07  1.040765e+09  5.077736e+07   
policy 3     7.032366e+07  7.021474e+07  8.110693e+07  7.021474e+07   
policy 4     5.038545e+08  1.858000e+07  2.785617e+09  1.858000e+07   
policy 5     3.897185e+08  1.858000e+07  2.719248e+09  1.858000e+07   
policy 6     5.038545e+08  1.858000e+07  2.785617e+09  1.858000e+07   
policy 7     3.897185e+08  1.858000e+07  2.719248e+09  1.858000e+07   
policy 8     4.933224e+08  1.858000e+07  2.785617e+09  1.858000e+07   
policy 9     4.933224e+08  1.858000e+07  2.785617e+09  1.858000e+07   

                        A.3_Expected Number of Deaths                      \
                    q95                          mean  min       max  q05   
policy                                                                      
policy 1   2.093779e+09                      0.644903  0.0  3.520692  0.0   
policy 10  1.671513e+09                      0.401836  0.0  3.236993  0.0   
policy 11  1.671513e+09                      0.401836  0.0  3.236993  0.0   
policy 12  1.671513e+09                      0.401836  0.0  3.236993  0.0   
policy 2   8.810651e+07                      0.046278  0.0  2.193363  0.0   
policy 3   7.021474e+07                      0.000261  0.0  0.026074  0.0   
policy 4   2.176020e+09                      0.840441  0.0  3.526107  0.0   
policy 5   2.093779e+09                      0.648298  0.0  3.520692  0.0   
policy 6   2.176020e+09                      0.840441  0.0  3.526107  0.0   
policy 7   2.093779e+09                      0.648298  0.0  3.520692  0.0   
policy 8   2.176020e+09                      0.822823  0.0  3.526107  0.0   
policy 9   2.176020e+09                      0.822823  0.0  3.526107  0.0   

                    RfR Total Costs                                         \
                q95            mean          min          max          q05   
policy                                                                       
policy 1   3.449067             0.0          0.0          0.0          0.0   
policy 10  3.183660      30700000.0   30700000.0   30700000.0   30700000.0   
policy 11  3.183660      30700000.0   30700000.0   30700000.0   30700000.0   
policy 12  3.183660      30700000.0   30700000.0   30700000.0   30700000.0   
policy 2   0.073038             0.0          0.0          0.0          0.0   
policy 3   0.000000             0.0          0.0          0.0          0.0   
policy 4   3.471726      84600000.0   84600000.0   84600000.0   84600000.0   
policy 5   3.449067             0.0          0.0          0.0          0.0   
policy 6   3.471726      84600000.0   84600000.0   84600000.0   84600000.0   
policy 7   3.449067             0.0          0.0          0.0          0.0   
policy 8   3.471726     217800000.0  217800000.0  217800000.0  217800000.0   
policy 9   3.471726     217800000.0  217800000.0  217800000.0  217800000.0   

                       Expected Evacuation Costs                      
                   q95                      mean  min  max  q05  q95  
policy                                                                
policy 1           0.0                       0.0  0.0  0.0  0.0  0.0  
policy 10   30700000.0                       0.0  0.0  0.0  0.0  0.0  
policy 11   30700000.0                       0.0  0.0  0.0  0.0  0.0  
policy 12   30700000.0                       0.0  0.0  0.0  0.0  0.0  
policy 2           0.0                       0.0  0.0  0.0  0.0  0.0  
policy 3           0.0                       0.0 

In [16]:
TCA_df.to_csv("results/TCA3_results.csv")